In [3]:
import sys
sys.path.append('..')
import numpy as np
import cupy as cp
import dask.array as da
import jax.numpy as jnp

# Map

In [2]:
from pycsou.func.penalty import L1Norm, SquaredL2Norm
from pycsou.linop.conv import Convolve1D

for xp in [np, cp]: #, da, jnp]:
    x = xp.arange(10)
    filter = np.hanning(5)
    filter[filter.size//2:] = 0
    f1 = L1Norm(dim=x.size)
    f2 = SquaredL2Norm(dim=x.size)
    L1 = Convolve1D(size=x.size, filter=filter)
    L2 = Convolve1D(size=x.size, filter=filter/2)

    print(f1.shape, f2.shape, L1.shape, L2.shape)
    
    f3 = f1 / 3 + np.pi * f2
    print(xp.allclose(f3(x), f1(x) / 3 + np.pi * f2(x)))
    
    L3 = L1 * 3 - (L2 ** 2) / 6

    print(xp.allclose(L3(x), L1(x) * 3 - (L2(L2(x))) / 6))
    f4 = f3 * L3
    print(xp.allclose(f4(x), f3(L3(x))))
    
    print(xp.allclose(f3 * x, f3(x)))
    print(xp.allclose(f3 * x, f3 @ x))
    print(xp.allclose((f1 * L1)(x), (f1 @ L1)(x)))

    f5=f4.shifter(shift=2 * x)
    print(xp.allclose(f5(x), f4(x + 2 * x)))



(1, 10) (1, 10) (10, 10) (10, 10)
<pycsou.linop.base.HomothetyMap object at 0x7fcf868231d0> 3.141592653589793
<pycsou.func.penalty.SquaredL2Norm object at 0x7fcf6ee78850> [0 1 2 3 4 5 6 7 8 9]
<pycsou.linop.base.HomothetyMap object at 0x7fcf868231d0> 285.00000000000006
<pycsou.func.penalty.SquaredL2Norm object at 0x7fcf6ee78850> [0 1 2 3 4 5 6 7 8 9]
True
<pycsou.linop.base.HomothetyMap object at 0x7fcf86823350> 3
<pycsou.linop.base.HomothetyMap object at 0x7fcf86823650> 0.16666666666666666
<pycsou.linop.base.HomothetyMap object at 0x7fcf86823190> -1
<pycsou.linop.base.HomothetyMap object at 0x7fcf86823350> [0 1 2 3 4 5 6 7 8 9]
<pycsou.linop.base.HomothetyMap object at 0x7fcf86823190> [0 1 2 3 4 5 6 7 8 9]
<pycsou.linop.base.HomothetyMap object at 0x7fcf86823650> [ 0 -1 -2 -3 -4 -5 -6 -7 -8 -9]
True
<pycsou.linop.base.HomothetyMap object at 0x7fcf86823350> [0 1 2 3 4 5 6 7 8 9]
<pycsou.linop.base.HomothetyMap object at 0x7fcf86823190> [0 1 2 3 4 5 6 7 8 9]
<pycsou.linop.base.Homothety

## apply_along_axis

In [3]:
from pycsou.linop import DownSampling
from pycsou.func import SquaredL2Norm

for xp in [np, cp]: #, da, jnp]:
    D=DownSampling(size=20, downsampling_factor=2)
    arr=xp.arange(200).reshape(20,2,5)
    out = D.apply_along_axis(arr, axis=0)
    print(out.shape)
    ff = SquaredL2Norm(dim=2)
    out = ff.apply_along_axis(arr, axis=1)
    print(out.shape)

<pycsou.linop.sampling.DownSampling object at 0x7fcf6ee78610> [[[  0   1   2   3   4]
  [  5   6   7   8   9]]

 [[ 10  11  12  13  14]
  [ 15  16  17  18  19]]

 [[ 20  21  22  23  24]
  [ 25  26  27  28  29]]

 [[ 30  31  32  33  34]
  [ 35  36  37  38  39]]

 [[ 40  41  42  43  44]
  [ 45  46  47  48  49]]

 [[ 50  51  52  53  54]
  [ 55  56  57  58  59]]

 [[ 60  61  62  63  64]
  [ 65  66  67  68  69]]

 [[ 70  71  72  73  74]
  [ 75  76  77  78  79]]

 [[ 80  81  82  83  84]
  [ 85  86  87  88  89]]

 [[ 90  91  92  93  94]
  [ 95  96  97  98  99]]

 [[100 101 102 103 104]
  [105 106 107 108 109]]

 [[110 111 112 113 114]
  [115 116 117 118 119]]

 [[120 121 122 123 124]
  [125 126 127 128 129]]

 [[130 131 132 133 134]
  [135 136 137 138 139]]

 [[140 141 142 143 144]
  [145 146 147 148 149]]

 [[150 151 152 153 154]
  [155 156 157 158 159]]

 [[160 161 162 163 164]
  [165 166 167 168 169]]

 [[170 171 172 173 174]
  [175 176 177 178 179]]

 [[180 181 182 183 184]
  [185 186 187

# Differentiable Map

In [3]:
from pycsou.func.penalty import SquaredL2Norm
from pycsou.linop.diff import FirstDerivative

for xp in [np, cp]: #, da, jnp]:
    x = xp.arange(10)
    f = SquaredL2Norm(dim=x.size)
    L1 = FirstDerivative(size=x.size)
    L1.compute_lipschitz_cst()
    L2 = FirstDerivative(size=x.size, kind='centered')
    L2.compute_lipschitz_cst()
    print(f.lipschitz_cst, f.diff_lipschitz_cst)
    print(xp.round(L1.lipschitz_cst,1), xp.round(L1.diff_lipschitz_cst,1))
    print(xp.round(L2.lipschitz_cst,1), xp.round(L2.diff_lipschitz_cst,1))
    L3 = 2 * L1 + (L2 ** 2) / 3
    print(xp.allclose(L3.lipschitz_cst, 2 * L1.lipschitz_cst + (L2.lipschitz_cst ** 2)/3))
    map_ = f * L1
    print(xp.allclose(map_.lipschitz_cst, f.lipschitz_cst * L1.lipschitz_cst))
    print(xp.allclose(map_.jacobianT(x), L1.jacobianT(x) * f.jacobianT(L1(x))))


inf 2
2.0 2.0
1.5 1.5
True
True
True
inf 2
2.0 2.0
1.5 1.5
True
True
True


# MapStack

In [4]:

from pycsou.func.penalty import SquaredL2Norm
from pycsou.linop.diff import FirstDerivative
from pycsou.core.map import MapStack

for xp in [np, cp]:
    x = xp.arange(10)
    y = xp.arange(20)
    f = SquaredL2Norm(dim=x.size)
    g = SquaredL2Norm(dim=y.size)
    A1 = FirstDerivative(size=x.size)
    A2 = FirstDerivative(size=y.size, kind='centered')
    A3 = A1 / 2
    K1 = f * A1
    K2 = g * A2
    K3 = A3

    V = MapStack(K1, K3, axis=0)
    print(V.shape)
    # (11, 10)
    print(xp.allclose(V(x), xp.concatenate((K1(x).flatten(), K3(x).flatten()))))
    # True
    parV = MapStack(K1, K3, axis=0, n_jobs=-1)
    print(xp.allclose(V(x), parV(x)))
    # True
    H = MapStack(K1,K2, axis=1)
    print(H.shape)
    # (1, 30)
    print(xp.allclose(H(xp.concatenate((x,y))), K1(x) + K2(y)))
    # True
    parH = MapStack(K1,K2, axis=1, n_jobs=-1)
    print(xp.allclose(H(xp.concatenate((x,y))), parH(xp.concatenate((x,y)))))
    # True


(11, 10)
True
True
(1, 30)
True
True
(11, 10)
True
True
(1, 30)
True
True


# MapVStack

In [5]:
from pycsou.core.map import MapVStack

for xp in [np, cp]:
    x = xp.arange(10)
    y = xp.arange(20)
    f = SquaredL2Norm(dim=x.size)
    g = SquaredL2Norm(dim=y.size)
    A1 = FirstDerivative(size=x.size)
    A2 = FirstDerivative(size=y.size, kind='centered')
    A3 = A1 / 2
    K1 = f * A1
    K2 = g * A2
    K3 = A3

    V = MapStack(K1, K3, axis=0)
    print(V.shape)
    # (11, 10)
    print(xp.allclose(V(x), xp.concatenate((K1(x).flatten(), K3(x).flatten()))))
    # True
    parV = MapStack(K1, K3, axis=0, n_jobs=-1)
    print(xp.allclose(V(x), parV(x)))
    # True
    H = MapStack(K1,K2, axis=1)
    print(H.shape)
    # (1, 30)
    print(xp.allclose(H(xp.concatenate((x,y))), K1(x) + K2(y)))
    # True
    parH = MapStack(K1,K2, axis=1, n_jobs=-1)
    print(xp.allclose(H(xp.concatenate((x,y))), parH(xp.concatenate((x,y)))))
    # True
    V1 = MapStack(K1, K3, axis=0)
    V2 = MapVStack(K1, K3)
    print(xp.allclose(V1(x), V2(x)))
    # True

(11, 10)
True
True
(1, 30)
True
True
True
(11, 10)
True
True
(1, 30)
True
True
True


# MapHStack

In [6]:
from pycsou.core.map import MapHStack

for xp in [np, cp]:
    x = xp.arange(10)
    y = xp.arange(20)
    f = SquaredL2Norm(dim=x.size)
    g = SquaredL2Norm(dim=y.size)
    A1 = FirstDerivative(size=x.size)
    A2 = FirstDerivative(size=y.size, kind='centered')
    A3 = A1 / 2
    K1 = f * A1
    K2 = g * A2
    K3 = A3

    V = MapStack(K1, K3, axis=0)
    print(V.shape)
    # (11, 10)
    print(xp.allclose(V(x), xp.concatenate((K1(x).flatten(), K3(x).flatten()))))
    # True
    parV = MapStack(K1, K3, axis=0, n_jobs=-1)
    print(xp.allclose(V(x), parV(x)))
    # True
    H = MapStack(K1,K2, axis=1)
    print(H.shape)
    # (1, 30)
    print(xp.allclose(H(xp.concatenate((x,y))), K1(x) + K2(y)))
    # True
    parH = MapStack(K1,K2, axis=1, n_jobs=-1)
    print(xp.allclose(H(xp.concatenate((x,y))), parH(xp.concatenate((x,y)))))
    # True
    V1 = MapStack(K1, K2, axis=1)
    V2 = MapHStack(K1, K2)
    print(xp.allclose(V1(np.arange(V1.shape[1])), V2(np.arange(V1.shape[1]))))
    #True

(11, 10)
True
True
(1, 30)
True
True
True
(11, 10)
True
True
(1, 30)
True
True
True


# DiffMapStack

In [4]:

from pycsou.func.penalty import SquaredL2Norm
from pycsou.linop.diff import FirstDerivative
from pycsou.core.map import MapStack, DiffMapStack

for xp in [np,cp]:
    print(xp)
    x = xp.arange(10)
    y = xp.arange(20)
    f = SquaredL2Norm(dim=x.size)
    g = SquaredL2Norm(dim=y.size)
    A1 = FirstDerivative(size=x.size)
    A1.compute_lipschitz_cst()
    A2 = FirstDerivative(size=y.size, kind='centered')
    A2.compute_lipschitz_cst()
    A3 = A1 / 2
    K1 = f * A1
    K2 = g * A2
    K3 = A3
    V = DiffMapStack(K1, K3, axis=0)
    print(xp.allclose(V.lipschitz_cst, xp.sqrt(K1.lipschitz_cst ** 2 + K3.lipschitz_cst ** 2)))
    # True
    parV = DiffMapStack(K1, K3, axis=0, n_jobs=-1)
    VV = V.jacobianT(x) * xp.ones(shape=(V.jacobianT(x).shape[1]))
    # print('VV;', VV)
    # print('parV.jac:', parV.jacobianT(x))
    # print('ones: ',xp.ones(shape=(V.jacobianT(x).shape[1])))
    
    # print(VV,  xp.ones(shape=(V.jacobianT(x).shape[1]))* parV.jacobianT(x))
    print(xp.allclose(VV, parV.jacobianT(x) * xp.ones(shape=(V.jacobianT(x).shape[1]))))

    # True

<module 'numpy' from '/home/andresmntr/anaconda3/envs/pycsou37/lib/python3.7/site-packages/numpy/__init__.py'>
True


BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [ ]:
# https://stackoverflow.com/questions/56154654/a-task-failed-to-un-serialize
# You should not have such an error with the last version of scikit-learn.
# This always happens when using multiprocessing in an iPython console in Spyder. A workaround is to run the script from the command line instead.